In [1]:
import pandas as pd
import numpy as np

In [23]:
file_path = "otodom/geocoded/otodom_geocoded.csv"
df = pd.read_csv(file_path, sep="\t")
df.shape

(34931, 34)

In [24]:
df[df.duplicated()].shape

(262, 34)

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
df.rename(columns = {"Url": "URL"}, inplace=True)

In [5]:
df["Cena"] = df["Cena"].str[:-3]
df["Cena"] = df["Cena"].str.replace(",", ".")
df["Cena"] = df["Cena"].str.replace(" ", "")
df["Cena"] = df["Cena"].astype(float)

In [6]:
df["Powierzchnia"] = df["Powierzchnia"].str[:-3]
df["Powierzchnia"] = df["Powierzchnia"].str.replace(",", ".")
df["Powierzchnia"] = df["Powierzchnia"].str.replace(" ", "")
df["Powierzchnia"] = df["Powierzchnia"].astype(float)

In [7]:
df["Liczba pokoi"].replace({"wiecej niz 10": 15}, inplace=True)
df["Liczba pokoi"] = df["Liczba pokoi"].astype(float)

In [8]:
from typing import Optional

def convert_string_to_floor(floor: str, max_floor: Optional[str] = None):
    if floor == "parter":
        return 0
    elif floor == "suterena":
        return -1
    elif floor == "poddasze":
        return int(max_floor)
    else:
        return int(floor)

def prepare_floor(row: pd.Series):
    floor = row["Pietro"]
    floor = floor.replace(">", "")
    floor_elements = floor.split("/")
    if len(floor_elements) == 2:
        your_floor = floor_elements[0].strip()
        max_floor = floor_elements[1].strip()
        your_floor = convert_string_to_floor(your_floor, max_floor)
        max_floor = int(max_floor)
           
    if len(floor_elements) == 1:
        your_floor = floor_elements[0].strip()
        if your_floor == "Zapytaj":
            your_floor = None
            max_floor = None
        else:
            your_floor = convert_string_to_floor(your_floor)
        max_floor = None
    
    row["pietro mieszkania"] = your_floor
    row["maksymalne pietro"] = max_floor
    return row
    
df = df.apply(prepare_floor, axis=1)

In [9]:
df["polozenie mieszkania"] = df["pietro mieszkania"] / df["maksymalne pietro"]

In [10]:
df["Czynsz"] = df["Czynsz"].str.replace("zl", "")
df["Czynsz"] = df["Czynsz"].str.replace("EUR", "")
df["Czynsz"] = df["Czynsz"].str.replace(",", ".")
df["Czynsz"] = df["Czynsz"].str.replace(" ", "")
df.loc[(df["Czynsz"] == "Zapytaj") | (df["Czynsz"].isna()), "Czynsz"] = np.nan
df["Czynsz"] = df["Czynsz"].astype(float)


In [11]:
df.loc[df["Stan wykonczenia"] == "Zapytaj", "Stan wykonczenia"] = None
df.loc[df["Stan wykonczenia"].isna(), "Stan wykonczenia"] = None

In [12]:
df.rename(columns = {"Balkon / ogrod / taras": "Powierzchnia dodatkowa"}, inplace=True)
df.loc[df["Powierzchnia dodatkowa"] == "Zapytaj", "Powierzchnia dodatkowa"] = 0
df.loc[df["Powierzchnia dodatkowa"] != 0, "Powierzchnia dodatkowa"] = 1

In [13]:
df.loc[(df["Miejsce parkingowe"] == "Zapytaj")|(df["Miejsce parkingowe"].isna()), "Miejsce parkingowe"] = 0
df.loc[(df["Miejsce parkingowe"] != 0), "Miejsce parkingowe"] = 1

In [14]:
df.loc[(df["Rynek"] == "Zapytaj")|(df["Rynek"].isna()), "Rynek"] = None

In [15]:
df.loc[(df["Rok budowy"] == "brak informacji") | (df["Rok budowy"].isna()), "Rok budowy"] = None

In [16]:
df.loc[(df["Winda"] == "nie") | (df["Winda"].isna()), "Winda"] = 0
df.loc[df["Winda"] == "tak", "Winda"] = 1
df["Winda"] = df["Winda"].astype(float)

In [17]:
df.drop(columns=[
    "Adres",
    "Tytul",
    "Obsluga zdalna",
    "Forma wlasnosci",
    "Ogrzewanie",
    "Typ ogloszeniodawcy",
    "Dostepne od",
    "Rodzaj zabudowy", 
    "Okna",
    "Media",
    "Zabezpieczenia",
    "Wyposazenie",
    "Informacje dodatkowe",
    "Material budynku"
], inplace=True)

In [18]:
df["Dataset"] = "otodom"

In [19]:
df.columns

Index(['Miasto', 'URL', 'Cena', 'Powierzchnia', 'Liczba pokoi', 'Pietro',
       'Czynsz', 'Stan wykonczenia', 'Powierzchnia dodatkowa',
       'Miejsce parkingowe', 'Rynek', 'Rok budowy', 'Winda', 'latitude',
       'longitude', 'formatted_address', 'result_confidence', 'suburb',
       'building_category', 'result_type', 'pietro mieszkania',
       'maksymalne pietro', 'polozenie mieszkania', 'Dataset'],
      dtype='object')

In [20]:
df.shape

(34669, 24)

In [21]:
df.dtypes

Miasto                     object
URL                        object
Cena                      float64
Powierzchnia              float64
Liczba pokoi              float64
Pietro                     object
Czynsz                    float64
Stan wykonczenia           object
Powierzchnia dodatkowa     object
Miejsce parkingowe         object
Rynek                      object
Rok budowy                 object
Winda                     float64
latitude                  float64
longitude                 float64
formatted_address          object
result_confidence         float64
suburb                     object
building_category          object
result_type                object
pietro mieszkania         float64
maksymalne pietro         float64
polozenie mieszkania      float64
Dataset                    object
dtype: object

In [22]:
df.to_csv("otodom/otodom_before_merge.csv", index=False, sep="\t")